In [14]:
import tensorflow as tf
from tensorflow.keras import datasets
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical, load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from PIL import Image

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

import numpy as np

In [15]:
train_dir = '/home/galesky/Documents/GitHub/aaesongie/Lee/python/teukgang/img_danp/train'
test_dir = '/home/galesky/Documents/GitHub/aaesongie/Lee/python/teukgang/img_danp/test'

datagen = ImageDataGenerator(
    rescale=1./255
)


train_data = datagen.flow_from_directory(train_dir, target_size=(128, 128), batch_size=32, class_mode='binary')
test_data = datagen.flow_from_directory(test_dir, target_size=(128, 128), batch_size=32, class_mode='binary')


x_train_list = []
y_train_list = []

for _ in range(train_data.samples // train_data.batch_size + 1):
    x_batch, y_batch = next(train_data)
    x_train_list.append(x_batch)
    y_train_list.append(y_batch)

x_train_all = np.concatenate(x_train_list)
y_train_all = np.concatenate(y_train_list)


x_test_list = []
y_test_list = []

for _ in range(test_data.samples // test_data.batch_size + 1):
    x_batch, y_batch = next(test_data)
    x_test_list.append(x_batch)
    y_test_list.append(y_batch)

x_test = np.concatenate(x_test_list)
y_test = np.concatenate(y_test_list)


x_train, x_val, y_train, y_val = train_test_split(x_train_all, y_train_all, test_size=0.2, random_state=42)

Found 302 images belonging to 2 classes.
Found 80 images belonging to 2 classes.


ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.

In [2]:
print(f"x_train shape: {x_train.shape}")
print(f"x_val shape: {x_val.shape}")
print(f"x_test shape: {x_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_val shape: {y_val.shape}")
print(f"y_test shape: {y_test.shape}")

sample_image = x_train[0]
sample_label = y_train[0]

plt.imshow(sample_image)
plt.title(f"Label: {int(sample_label)}")
plt.axis('off')
plt.show()

print("Image shape:", sample_image.shape)

print("Label:", int(sample_label))

NameError: name 'x_train' is not defined

In [ ]:
def model_b0():
    model = Sequential()

    input = Input(shape=(128, 128, 3))

    x = Flatten()(input)
    x = Dense(256, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    x = Dense(32, activation='relu')(x)

    output = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=[input], outputs=output)

    model.compile(optimizer=Adam(learning_rate=1e-4),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])


    return model


model = model_b0()

model.summary()

In [ ]:
model = model_b0()

history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=10,
    )


test_loss, test_acc = model.evaluate(x_test, y_test, verbose = 2)
print(f"테스트 정확도: {test_acc:.4f}")

In [ ]:
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(history.history['loss'], 'b--', label='loss')
plt.plot(history.history['val_loss'], 'r:', label='val loss')
plt.xlabel('Epochs')
plt.grid()
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], 'b--', label='accuracy')
plt.plot(history.history['val_accuracy'], 'r:', label='val accuracy')
plt.xlabel('Epochs')
plt.grid()
plt.legend()

In [ ]:
model.save('/home/galesky/Documents/GitHub/aaesongie/Lee/python/teukgang/models/1.keras')

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, BatchNormalization
from tensorflow.keras.applications import VGG16

In [ ]:
def model_b1():
    model = Sequential()

    input = Input(shape=(128, 128, 3))

    x = Conv2D(filters=32, kernel_size=3, padding='same', activation='relu')(input)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(2, 2), strides=2, padding='same')(x)

    x = Conv2D(filters=64, kernel_size=3, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(2, 2), strides=2, padding='same')(x)

    x = Conv2D(filters=128, kernel_size=3, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters=128, kernel_size=3, padding='same', activation='relu')(x)
    x = MaxPool2D(pool_size=(2, 2), strides=2, padding='same')(x)
    x = Dropout(0.4)(x)

    x = Conv2D(filters=256, kernel_size=3, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters=256, kernel_size=3, padding='same', activation='relu')(x)
    x = MaxPool2D(pool_size=(2, 2), strides=2, padding='same')(x)
    x = Dropout(0.5)(x)

    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.6)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)

    output = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=[input], outputs=output)

    model.compile(optimizer=Adam(learning_rate=1e-4),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

In [ ]:
model = model_b1()

history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=60
    )

test_loss, test_acc = model.evaluate(x_test, y_test, verbose = 2)
print(f"테스트 정확도: {test_acc:.4f}")

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

for layer in base_model.layers:
    layer.trainable = False

def model_b2():
    model = Sequential()

    input = Input(shape=(128, 128, 3))

    x = base_model(input)

    x = Conv2D(filters=32, kernel_size=3, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(2,2), strides=2, padding='same')(x)
    x = Dropout(0.3)(x)

    x = Conv2D(filters=64, kernel_size=3, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(2,2), strides=2, padding='same')(x)
    x = Dropout(0.5)(x)

    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)

    output = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=[input], outputs=output)

    model.compile(optimizer=Adam(learning_rate=1e-4),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])


    return model

In [ ]:
model = model_b2()

history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=60
    )

test_loss, test_acc = model.evaluate(x_test, y_test, verbose = 2)
print(f"테스트 정확도: {test_acc:.4f}")

In [ ]:
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(history.history['loss'], 'b--', label='loss')
plt.plot(history.history['val_loss'], 'r:', label='val loss')
plt.xlabel('Epochs')
plt.grid()
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], 'b--', label='accuracy')
plt.plot(history.history['val_accuracy'], 'r:', label='val accuracy')
plt.xlabel('Epochs')
plt.grid()
plt.legend()

In [ ]:
model.save('/home/galesky/Documents/GitHub/aaesongie/Lee/python/teukgang/models/2.keras')